In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   age               28 non-null     int64
 1   affordibility     28 non-null     int64
 2   bought_insurance  28 non-null     int64
dtypes: int64(3)
memory usage: 800.0 bytes


### EDA

In [4]:
df.isnull().sum()

age                 0
affordibility       0
bought_insurance    0
dtype: int64

### Input and target variable

In [5]:
x = df[['age','affordibility']]
y = df['bought_insurance']

### Split Train & Test Data

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=25)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((22, 2), (6, 2), (22,), (6,))

#### Scale the data

In [7]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

<img src="nn.png" height=800 width=800/>

### Build the model

sigmoid(x) = 1 / (1 + exp(-x))

In [8]:
model = keras.Sequential([
    keras.layers.Dense(1,
                       input_shape=(2,),
                       activation='sigmoid',                              # sigmoid(x) = 1 / (1 + exp(-x))
                       kernel_initializer='ones',
                       bias_initializer='zeros')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


#### Compile

In [9]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

#### Train

In [10]:
model.fit(X_train_scaled, y_train, verbose=0, epochs=5000)

#### Evaluate the model

In [11]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 289ms/step - loss: 0.3550 - accuracy: 1.0000


[0.35497748851776123, 1.0]

#### Predict

In [12]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 144ms/step


array([[0.70548487],
       [0.35569552],
       [0.16827849],
       [0.47801173],
       [0.7260697 ],
       [0.82949835]], dtype=float32)

In [13]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

**weights and bias from the model**

In [14]:
coef, intercept = model.get_weights()
print('Weight :',coef)
print('bias',intercept)

Weight : [[5.060867 ]
 [1.4086502]]
bias [-2.9137027]


**This means w1=5.060867, w2=1.4086502, bias =-2.9137027**

**Instead of model.predict, write our own prediction function that uses w1,w2 and bias**

In [15]:
def sigmoid(X):
    return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

NameError: name 'sigmoid_numpy' is not defined

In [ ]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

prediction_function(.47, 1)

In [ ]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [ ]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)

    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_predicted)
        

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d
        

        #print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')


        if loss<=loss_thresold:
            break


    return w1, w2, bias

In [ ]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000, 0.4631)

In [ ]:
coef, intercept